# Lazy Processing and Pipelines

In your homework this last week you saw the example of chaining together iterators ti implement a pipeline. Despite the feeling that you were doing a pipeline, there was an outside enveloping inside, not a true pipeline feel to what u did. The averaging had to wrap the generation, for example.

It wasnt as if these were separate processes. Everything was co-ordinated by the wrapper. In other words, the programming style was bad, our iterators were not independent and reusable in a pipeline.

We'll start to remedy this here.

We'll first talk about generators and streams, and more generally about lazy processing. Then we'll see how to arrange our computation into "independent processes".

## Generator functions, redux

In [3]:
class Sentence:#an iterable
    def __init__(self, text): 
        self.text = text
        self.words = text.split()
        
    def __iter__(self):#one could also return iter(self.words)
        for w in self.words:#note this is implicitly making an iter from the list
            yield w
    
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

In [4]:
a = Sentence("the mad dog went home to his cat")

In [5]:
for w in a:
    print(w)

the
mad
dog
went
home
to
his
cat


## Lazy processing

Upto now, it might just seem that we have just represented existing sequences in a different fashion. But notice above, with the use of `yield`, that we do not have to define the entire sequence ahead of time. Indeed we talked about this a bit when we talked about iterators, but we can see this "lazy behavior" more explicitly now. We see it in the generation of infinite sequences, where there is no data per se!

So, because of generators, we can go from fetching items from a collection to "generate"ing iteration over arbitrary, possibly infinite series...

In [6]:
def fibonacci(): 
    i,j=0,1 
    while True: 
        yield j
        i,j=j,i+j

In [7]:
f = fibonacci()
for i in range(10):
    print(next(f))

1
1
2
3
5
8
13
21
34
55


In [8]:
f = fibonacci()
counter = 0
for i in f:#calls iter on itself but thats ok.
    print(i)
    counter += 1
    if counter > 9:
        break

1
1
2
3
5
8
13
21
34
55


### Lazy Evaluation and Streams

The basic idea in progressions such as the above is that the next element in the progression (the next fibonacci) is not computed until requested. This can also be achieved via **Streams**.

A stream is a lazily computed linked list. Remember how we created a first and a rest in linked lists? now we do the same, except that we compute the "rest" only when we access it. Basically we store a function to compute, and the state of our iteration, and compute-on-access

These examples are taken from "Composing Programs" (the successor to cs61a):

In [9]:
class Link:
        """A linked list with a first element and the rest."""
        empty = ()
        def __init__(self, first, rest=empty):
            assert rest is Link.empty or isinstance(rest, Link)
            self.first = first
            self.rest = rest
        def __getitem__(self, i):
            if i == 0:
                return self.first
            else:
                return self.rest[i-1]
        def __len__(self):
            return 1 + len(self.rest)

In [10]:
s = Link(3, Link(4, Link(5)))
len(s), s[1]

(3, 4)

In [11]:
class Stream:
    """A lazily computed linked list."""
    class empty:
        def __repr__(self):
            return 'Stream.empty'
    
    def __init__(self, first, compute_rest=empty):
        assert callable(compute_rest), 'compute_rest must be callable.'
        self.first = first
        self._compute_rest = compute_rest
        
    @property
    def rest(self):
        """Return the rest of the stream, computing it if necessary."""
        if self._compute_rest is not None:
            self._rest = self._compute_rest()
            self._compute_rest = None
        return self._rest
    
    def __repr__(self):
        return 'Stream({0}, <...>)'.format(repr(self.first))

In [12]:
s = Stream(0, lambda: Stream(2, lambda: Stream(4)))
s

Stream(0, <...>)

In [16]:
print(s.first, "\n",
s.rest, "\n",
s.rest.first, "\n", 
s.rest.rest, "\n", 
s.rest.rest.first, "\n", 
s.rest.rest.rest)
s.first, s.rest, s.rest.first, s.rest.rest

0 
 Stream(2, <...>) 
 2 
 Stream(4, <...>) 
 4 
 Stream.empty


(0, Stream(2, <...>), 2, Stream(4, <...>))

In [14]:
def get_evens(start):
    def compute_rest():
        return get_evens(start + 2)
    return Stream(start, compute_rest)

In [15]:
e=get_evens(0)
e.first, e.rest, e.rest.first, e.rest.rest

(0, Stream(2, <...>), 2, Stream(4, <...>))

The same thing can be done with a generator function:

In [17]:
def progression(begin, end=None): #could be done as __iter__ in a class
    result = begin 
    forever = end is None
    while forever or result < end: 
        yield result
        result = result +2
ag = progression(0, 20)
list(ag)

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

In [18]:
ag

<generator object progression at 0x1053ce8e0>

### But we do have state

A generator is stateful, has a point in the iteration, but a stream does not. On the other hand a stream uses a lot of the call stack

In [19]:
next(ag)

StopIteration: 

We'd have to recreate the generator to use it again...

Notice the stream does not have state like the generator: the same instance can be used again

In [20]:
e.first, e.rest, e.rest.first, e.rest.rest

(0, Stream(2, <...>), 2, Stream(4, <...>))

The separation of whether we have state or not is the key thing that distinguishes both these ways of lazy implementation of computation: iterators follow the notions of iterative programs and have state (and thus we have the notion of exhausting an iterator) while streams have no state, and the computation happens (again and again) on the fly.

## Lazy Evaluation, more generally

Lazy evaluation is a deceptively simple concept: if I give you some expression like `(a+b)*c`, normal "eager" languages will compute the value of the expression and return it.

A lazy evaluation of that expression doesn't return the value, it returns a thing that represents what the answer will be, should you ever ask for it. This is the tack a language like Haskell uses, as opposed to ML, which are both in one family of statically typed functional languages (check out Elm, in the same family which compiles down for javascript on the web, and which implements streams as a primitive).

But *until* you actually ask that thing what the answer was, no computation is actually done.

Why is this useful?
It can be for a number of reasons:

 - If you're not sure that you're actually going to need the answer, lazy evaluation avoids unnecessary computation.
 - It can be easier to express infinitely-large computations in lazy systems, since you can describe the computation without having to actually compute infinitely many values.
 - You can modify the expression before you actually evaluate it.

### A thunk class

As we mentioned, our lazy expressions are not going to return normal values.

For instance, adding two numbers won't return a number, it will return an object that represents the result of the computation, which you can ask for later.

This thing is sometimes called a [thunk](https://en.wikipedia.org/wiki/Thunk) or [future](https://en.wikipedia.org/wiki/Futures_and_promises).
While they can be implemented a number of ways, we're going to use a class.

Let us create a new class called `LazyOperation`, which will be our thunk.
The constructor will take one required argument `function` and then arbitrary positional and keyword arguments (this is the `*args` and `**kwargs` syntax, if you remember).

The constructor doesn't need to do anything but store them internally for now.

Eventually, when we do something like `(a+b)*c`, the value of the expression will end up being an instance of this thunk class.


### Recursive evaluation

You're probably wondering what use this is. How do we actually use these?

The answer is an `eval` method for `LazyOperation`.
This is the mechanism for us to ask for an actual answer.
In our example above:

```python
lazy_add(1,2).eval() == 3
```

`eval` takes no arguments: it has all the information it needs.
Its behavior is as follows:

 - First, transform the arguments: for all the positional and keyword arguments stored in the `LazyOperation`, if the argument is an instance of a `LazyOperation`, call `eval` on *it*. If it's anything else, do nothing.
 - Second, call the stored function on the transformed arguments and return the value.

That's it!

So what's actually going on here? Why do we need to recursively call `eval`? And why only on some arguments?

Let's use an example:

```python
thunk = lazy_mul( lazy_add(1,2), 4)
thunk.eval()
```

In the first line we are crafting a chain of `LazyOperation`s.
The expression `lazy_add(1,2)` returns an instance of `LazyOperation`. (1 and 2 are stored as its arguments.)
Then we create another thunk in `lazy_mul`, and the two stored arguments are the first `LazyOperation` instance and the number 4.
So something like this:

```python
thunk = LazyOperation( body_of_lazy_mul, LazyOperation( body_of_lazy_add, 1, 2 ), 4 )
```

Now we call `eval` on `thunk`.
`eval` will first recursively call `eval` on its `LazyOperation`-typed arguments (of which there is only one: the result of `lazy_add`).
This recursive invocation does the same, but it has no `LazyOperation` arguments, so it just calls its stored function (the body of the original `lazy_add` function: `a+b`) on its arguments 1 and 2.
The result of this is a number: 3.
Now the first `eval` function can complete, and it evaluates *its* stored function (the body of the original `lazy_mul` function) on its new, transformed arguments: 3 and 4.
This returns 7.

If you've done everything right, you should be able to run this example.

In [21]:
class LazyOperation():
    def __init__(self, function, *args, **kwargs):
        self._function = function
        self._args = args
        self._kwargs = kwargs

    def eval(self):
        # Recursively eval() lazy args
        new_args = [a.eval() if isinstance(a,LazyOperation) else a for a in self._args]
        new_kwargs = {k:v.eval() if isinstance(v,LazyOperation) else v for k,v in self._kwargs}
        return self._function(*new_args, **new_kwargs)

    # Debug:
    def thunk_tree(self, indent='| '):
        s = indent[:-2]+'| ['+self._function.__name__+']\n'
        for a in self._args:
            if isinstance(a, LazyOperation):
                s += a.thunk_tree(indent=indent+'| ')
            else:
                s += indent+'| '+str(a)+'\n'
        for k,v in self._kwargs:
            if isinstance(a, LazyOperation):
                s += str(k)+'='+v.thunk_tree(indent=indent+'| ')
            else:
                s += indent+'| '+str(k)+'='+str(v)+'\n'
        return s

### A lazy decorator

Now comes the interesting part.
We'd like to be able to turn *any* function lazy.
So let's create a temporary test file with the following:

```python
def add(a,b):
    return a+b
def mul(a,b):
    return a*b
```

We want to create a version of `add` which, instead of actually adding `a` and `b`, returns a `LazyOperation` thunk with represents what *would* happen if we added `a` and `b`.
This probably sounds harder than it is, and it actually doesn't involve rewriting `add`.

The trick is to realize that the notion of what the answer *would* be is the same as saying "don't evaluate this function just yet; instead, save it and all of its arguments so I can evaluate it later if I want."
This is one way of implementing lazy evaluation.

Let's be more clear:
you created a class which (right now) just stores a function and a set of arbitrary positional and keywords arguments.
This is exactly what you need to know if you want to evaluate that function later!
So we want to write a decorator which turns `add` into a function which returns a `LazyOperation` with all of its information stored inside, similar to this:

```python
@lazy
def lazy_add(a,b):
     return a+b

lazy_add(1,2) == LazyOperation( old_function, args=[a,b], kwargs={} )
```

Where `old_function` is the undecorated version of `lazy_add`.
(Remember that decorators are just functions which take a function as an argument, return a new function, and bind that new function to the original name.)

In [22]:
def lazy(function):
    def create_thunk(*args, **kwargs):
        return LazyOperation(function, *args, **kwargs)
    return create_thunk

In [23]:
@lazy
def lazy_add(a,b):
    return a+b

In [24]:
isinstance( lazy_add(1,2), LazyOperation ) == True

True

In [25]:
@lazy
def lazy_mul(a,b):
    return a*b

In [26]:
thunk = lazy_mul( lazy_add(1,2), 4)
thunk.eval()

12

The idea outlined here is to make all procedures take delayed arguments (thunks) which are forced (eval'ed) at some point. 

You can read about various evaluation models here:

https://en.wikipedia.org/wiki/Evaluation_strategy

## Lazy implementation for Sequences using generators

Despite all our talk of lazy implementation, our Sentence implementations so far have not been lazy because the __init__ eagerly builds a list of all words in the text, binding it to the self.words attribute. This will entail processing the entire text, and the list may use as much memory as the text itself

In [27]:
import re
WORD_REGEXP = re.compile('\w+')
class Sentence:#an iterable
    def __init__(self, text): 
        self.text = text
        
    def __iter__(self):
        for match in WORD_REGEXP.finditer(self.text):
            yield match.group()
    
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

In [28]:
list(Sentence("the mad dog went home to his cat"))

['the', 'mad', 'dog', 'went', 'home', 'to', 'his', 'cat']

### Generator Expressions of data sequences.

There is an even simpler way: use a generator expression, which is just a lazy version of a list comprehension. (itrs really just sugar for a generator function, but its a nice bit of sugar)


In [29]:
RE_WORD = re.compile('\w+')
class Sentence:#an iterable
    def __init__(self, text): 
        self.text = text
        
    def __iter__(self):
        return (match.group() for match in RE_WORD.finditer(self.text))
    
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
list(Sentence("the mad dog went home to his cat"))

['the', 'mad', 'dog', 'went', 'home', 'to', 'his', 'cat']

Which syntax to choose?

Write a generator function if the code takes more than 2 lines.

Some syntax that might trip you up: double brackets are not necessary

In [30]:
(i*i for i in range(5))

<generator object <genexpr> at 0x10599b6d0>

In [31]:
list((i*i for i in range(5)))

[0, 1, 4, 9, 16]

In [60]:
list(i*i for i in range(5))

[0, 1, 4, 9, 16]

### yield from

This syntax can be used to combine iterators

In [61]:
def mychain(*iterables):
    for it in iterables:
        for i in it: 
            yield i
        

In [62]:
def chain(*iterables):
    for it in iterables:
        yield from it# for i in it: yield i

In [63]:
s="ABC"
l=[1,2,3]
chain(s,l)

<generator object chain at 0x1059d10f8>

In [64]:
list(mychain(s,l))

['A', 'B', 'C', 1, 2, 3]

In [65]:
list(chain(s,l))

['A', 'B', 'C', 1, 2, 3]

## Concurrency and Coroutines

We briefly mentioned that Python has a GIL, which allows only one thread to run python code at a time. It was made to allow non-thread safe code to run, fast, in a single thread, which is a very common case.

When we run python code, we have no control over the GIL. But extension modules written in C can release the GIL while doing time-consuming tasks.  `numpy` does this, so for example, dot-products happen in a C context outside the GIL.

All the standard library functions that do blocking IO also release the GIL, so that python code can continue to run while waiting for network-IO. This means that if you use threads or some other concurrency mechanism, while one unit of concurrency is waiting for a response from the netweork or a read from disk, another unit can continue doing something CPU-bound. 

>David Beazley says: “Python threads are great at doing nothing.”

The time.sleep() function also releases the GIL. The whole thing looks a bit like this:

```c
Py_BEGIN_ALLOW_THREADS
        sleep((int)secs);
Py_END_ALLOW_THREADS
```

We'll see threads and processes later. But threads take 50k of memory per thread and are expensive to switch to, and the OS has a hard limit on threads. Instead, we we could create granular units of processing and IO-checking an IO-doing within the process, we could have our one thread running at full capacity rather than waiting on IO, at a low overhead.

Whats our primitive for yielding control of execution? Why `yield` and generators, of-course.

A generator becomes a co-routine when you can send data into a generator. This is done by adding a new primitive to generators after `next`: `.send`. Python can start hundreds of thousands of coroutines.

Thus coroutines are "subroutines" that can pause and resume. Threads are scheduled by the OS, but co-routines multi-task co-operatively.

In Python 3.5 native coroutine support was also added using `async` and `await`, however, " understanding coroutines as they were first implemented in Python 3.4, using pre-existing language facilities, is the foundation to tackle Python 3.5's native coroutines." (from Guido in http://aosabook.org/en/500L/a-web-crawler-with-asyncio-coroutines.html )

We will achieve "concurrency" here, not "parallelism" (such as the map-reduce pattern). It is designed for I/O-bound problems, not CPU-bound ones. Specifically this is true for multiple sleepy or slow connections, as in web crawling. 

### Generators with yield

We have so far used generators to implement iterators with the `yield` keyword. The iteration happens via a driver program using `next` to advance the generator

Lets start with a very simple example of a generator, which we see below:

In [32]:
def cr0(y):
    print("y>",y)
    yield y
    print("2y>",2*y)
    yield y+y
    print("3y>",3*y)
    yield y+y+y

In [33]:
cr0i = cr0(5)
list(cr0i)

y> 5
2y> 10
3y> 15


[5, 10, 15]

In [34]:
next(cr0i)

StopIteration: 

### From generators to co-routines

Now, lets make two changes. Lets assign, in the generator, to the result of the yield, and use a new method on the generator, `send`, in addition to the `next` function (or `__next__` method) that we have been using

In [35]:
from inspect import getgeneratorstate

In [40]:
def cr(y):
    print("y>",y)
    x = yield y+1
    print("x>",x)
    z = yield x+y
    print("z>",z)
    return z

Notice we "assign" x to the yield of y.

In [41]:
cri=cr(5)
type(cri)

generator

In [42]:
getgeneratorstate(cri)

'GEN_CREATED'

In [43]:
a=next(cri)#advance to first yield and get y+1 out
a

y> 5


6

In [44]:
getgeneratorstate(cri)

'GEN_SUSPENDED'

The syntax `x = yield y` is not what it seems. `yield` suspends the generator, sending the value of `y` out as 5. Now, the generator can be advanced using either `next` or `.send`. The latter allows us to send INTO the generator a value, which can then be used as x

In [45]:
b = cri.send(66) #at first yield send 66 in, advance to second yield, 

x> 66


In [46]:
b # the value from second yield is saved out

71

We have advanced to the next `yield`, which has yielded thse sum of 5 and the sent-in 6.

In [47]:
getgeneratorstate(cri)

'GEN_SUSPENDED'

Now we send in 22 as `z`. There are no more yields, and we come to the end, ie to a `StopIteration`. But notice now that we have a `return`. This return is now sent out as the value of the exception.

In [48]:
cri.send(22) # send in and fall of cliff as no more yields

z> 22


StopIteration: 22

In [14]:
getgeneratorstate(cri)

'GEN_CLOSED'

What if we caused an exception earlier by sending in a non-number?

In [72]:
def cr2(y):
    print("y>",y)
    x = yield y+1
    print("x>",x)
    z = yield x+y
    print("z>",z)
    a = yield z
    print("a>",a)
    return z
cri=cr2(5)

In [73]:
next(cri)# advance to first yield

y> 5


6

In [50]:
next(cri) # equivalent to cri.send(None)

x> None


TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'

Aha! `next` is equivalent to `.send(None)`. But the exception kills the generator and we cannot go any further...

In [51]:
getgeneratorstate(cri)

'GEN_CLOSED'

In [52]:
cri.send(None)

StopIteration: 

**This generator, with `yield` appearing on the right side of an expression, is a coroutine**.

- the yield is a RHS
- we could have just `yield`, which is equivalent to `yield None`
- the coroutine uses yield to send data out to the caller or driver
- which uses assignment to the yield in conjunction with `send` to send a value ointo the coroutine. Sending None is equivalent to `next`
- thus one may yield nothing, and send in nothing. Still, the generator has been paused...

In this sense, `yield` is a **control-flow** device, and it can be used to implement co-operative multi-tasking (like in the old iphones): *each coroutine yields to a central driver which can then re-activate it or other couroutines by appropriate sends*.

Think of **yield** as a control-flow device from now on!

These changes came into Python incrementally. From Fluent

>The infrastructure for coroutines appeared in PEP 342 — Coroutines via Enhanced Generators, implemented in Python 2.5 (2006): since then, the yield keyword can be used in an expression, and the .send(value) method was added to the generator API. Using .send(...), the caller of the generator can post data that then becomes the value of the yield expression inside the generator function. This allows a generator to be used as a coroutine: a procedure that collaborates with the caller, yielding and receiving values from the caller.

>In addition to .send(...), PEP 342 also added .throw(...) and .close() methods that respectively allow the caller to throw an exception to be handled inside the generator, and to terminate it. 

>The latest evolutionary step for coroutines came with PEP 380 - Syntax for Delegating to a Subgenerator, implemented in Python 3.3 (2012). PEP 380 made two syntax changes to generator functions, to make them more useful as coroutines:
- A generator can now return a value; previously, providing a value to the return statement inside a generator raised a SyntaxError.
- The `yield from` syntax enables complex generators to be refactored into smaller, nested generators while avoiding a lot of boilerplate code previously required for a generator to delegate to subgenerators.



### More coroutines

So far, we have seen the use of `next` and `send`, and the consequences of `send`ing something of the wrong type (we exhaust the generator). Lets explore these, `throw`, and `close`, in the context of a more useful generator, one which keeps yielding  us a running average.

In [53]:
def averager(): 
    total = 0.0
    count = 0 
    average = None 
    while True:
        term = yield average 
        total += term
        count += 1
        average = total/count

In [54]:
av=averager()
getgeneratorstate(av)

'GEN_CREATED'

In [55]:
print(next(av))#advance to first yield
getgeneratorstate(av)

None


'GEN_SUSPENDED'

In [56]:
print(av.send(5))
print(av.send(4))
print(av.send(3))
print(getgeneratorstate(av))

5.0
4.5
4.0
GEN_SUSPENDED


#### `close`

Since this generator is in an infinite loop, we must close it. We can do it cleanly by `.close()` which sends in a `GeneratorExit` exception which is implicitly handled by generators.

In [57]:
av.close()
print(getgeneratorstate(av))

GEN_CLOSED


#### `throw`

We can throw in any exceptionexplicitly using `.throw`. If the generator handles the exception (we can write code to do so) we are advanced to the next yield. Othwewise the generator is exhausted, and the exception propagates back out

In [59]:
av=averager()
next(av)#advance to first yield
av.throw(ValueError("yes"))

ValueError: yes

In [60]:
av=averager()
next(av)#advance to first yield
av.throw(GeneratorExit())

GeneratorExit: 

In [61]:
print(getgeneratorstate(av))

GEN_CLOSED


Here we handle `ValueError`: the exception handling code runs and advances us to the next yield.

In [62]:
def averager2(): 
    total = 0.0
    count = 0 
    average = None 
    while True:
        try:
            term = yield average
        except ValueError:
            term=0
        total += term
        count += 1
        average = total/count

In [63]:
av=averager2()
next(av)#advance to first yield
av.throw(ValueError("yes"))

0.0

In [64]:
av.send(5)

2.5

In [65]:
av.close()

#### Returning a result

Notice below now that we dont yield a running average, instead doing it as the return. So now we must create a termination condition. as you can see, if we send in a None, we achieve this by breaking the loop.

In [74]:
from collections import namedtuple
Result = namedtuple('Result', 'count average')
def averager(): 
    total = 0.0
    count = 0 
    average = None 
    while True:
        term = yield #new syntx, average is not yielded out
        print("Term is", term)
        if term is None:
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average)

In [75]:
av=averager()

In [76]:
next(av)
av.send(5)

Term is 5


In [77]:
av.send(3)

Term is 3


In [78]:
next(av)#seems to be same as av.send(None)

Term is None


StopIteration: Result(count=2, average=4.0)

It might seem strange that the return value of the coroutine is sent back on the exception. The reason for this is that we want to preserve the semantics of generator objects which come to us from a totally different use case of iteration: the raising of `StopIteration` when exhausted.

This will be important a bit later when we see `yield from`. `StopIteration` is handled transparently in loops; similarly, in `yield from` this also happens: indeed we consume the `StopIteration`, and the value sent on it: the result, becomes the ultimate value of the `yield from` expression itself.

In [79]:
def averager_subgen():
    print("    |sg>starting subgen")
    total = 0.0
    count = 0 
    average = None 
    while True:
        print("    |sg>  average yielded out", average)
        term = yield average
        print("    |sg>  term sent in", term)
        if term is None:
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average)

In [81]:
def average_my_values_delegen_simple():
    agen = averager_subgen()
    print("  |dg>created a new averager",id(agen))
    overall_av = yield from agen
    #yield from consumes all the values, like a list
    #not an individual value
    print("  |dg>now", overall_av)
    return overall_av

In [82]:
values_to_send=[1,2,3,4,5,6]

In [89]:
print("1. creating delegating generator")
delegating_gen = average_my_values_delegen_simple()
print(type(delegating_gen))
print("2. priming till yield from by sending in None")
next(delegating_gen)#priming
print("3. in loop after first yield, None sent in")
for value in values_to_send:
    print(">>sending term",value)
    out = delegating_gen.send(value)
    print('<<getting running average', out)
print("4. Sending in None to terminate")
out = delegating_gen.send(None)
print('<<getting running average', out)
print("5. DONE")

1. creating delegating generator
<class 'generator'>
2. priming till yield from by sending in None
  |dg>created a new averager 4388926392
    |sg>starting subgen
    |sg>  average yielded out None
3. in loop after first yield, None sent in
>>sending term 1
    |sg>  term sent in 1
    |sg>  average yielded out 1.0
<<getting running average 1.0
>>sending term 2
    |sg>  term sent in 2
    |sg>  average yielded out 1.5
<<getting running average 1.5
>>sending term 3
    |sg>  term sent in 3
    |sg>  average yielded out 2.0
<<getting running average 2.0
>>sending term 4
    |sg>  term sent in 4
    |sg>  average yielded out 2.5
<<getting running average 2.5
>>sending term 5
    |sg>  term sent in 5
    |sg>  average yielded out 3.0
<<getting running average 3.0
>>sending term 6
    |sg>  term sent in 6
    |sg>  average yielded out 3.5
<<getting running average 3.5
4. Sending in None to terminate
    |sg>  term sent in None
  |dg>now Result(count=6, average=3.5)


StopIteration: Result(count=6, average=3.5)